In [6]:
# import libraries
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# set up plotly in 'offline' mode
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
print (__version__) # requires version >= 1.9.0

2.2.1


In [12]:
import plotly.offline as plotly  ##testing
import numpy as np

In [13]:
# quick example (works)

x=np.array([2,5,8,0,2,-8,4,3,1])
y=np.array([2,5,8,0,2,-8,4,3,1])

data = [go.Scatter(x=x,y=y)]
fig = go.Figure(data = data,layout = go.Layout(title='Offline Plotly Testing',width = 800,height = 500,
                                           xaxis = dict(title = 'X-axis'), yaxis = dict(title = 'Y-axis')))

#plotly.offline.iplot(data)
plotly.offline.iplot(fig)

In [14]:
import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv")

table = ff.create_table(df)
#plotly.offline.iplot(table, filename='jupyter/table1')                           #filename does what here?
plotly.offline.iplot(table)

In [15]:
data = [go.Bar(x=df.School,
            y=df.Gap)]

plotly.offline.iplot(data, filename='jupyter/basic_bar')

In [16]:
from plotly.graph_objs import *  #testing

trace_women = Bar(x=df.School,
                  y=df.Women,
                  name='Women',
                  marker=dict(color='#ffcdd2'))

trace_men = Bar(x=df.School,
                y=df.Men,
                name='Men',
                marker=dict(color='#A2D5F2'))

trace_gap = Bar(x=df.School,
                y=df.Gap,
                name='Gap',
                marker=dict(color='#59606D'))

data = [trace_women, trace_men, trace_gap]
layout = Layout(title="Average Earnings for Graduates",
                xaxis=dict(title='School'),
                yaxis=dict(title='Salary (in thousands)'))
fig = Figure(data=data, layout=layout)

#plotly.offline.iplot(fig, filename='jupyter/styled_bar')
plotly.offline.iplot(data, layout)

In [17]:
#create Dataframe for all settlement-results, which will be calculated 

DF_all = pd.DataFrame()

# import the DataFrame with calculated Results from GIS
tv_df = pd.read_csv('Ostroehre.TunnelExcavationData.R2.csv',index_col=0)

# calculate settlements at stationed points along tunnel alignment
len_df = len(tv_df.index) - 1   #how many points are there -> length of loop
for i in range(len_df): #start loop
    a=array( [tv_df['Easting'][i], tv_df['Northing'][i], tv_df['Elevation'][i], tv_df['DTM'][i], tv_df['RockCover'][i]]) # read in the values which are needed for calculation of point i
    a=asfarray(a,float)
    b=array( [tv_df['Easting'][i+1], tv_df['Northing'][i+1], tv_df['Elevation'][i+1], tv_df['DTM'][i+1], tv_df['RockCover'][i+1]] ) # read in the values which are needed for calculation of point i+1
    b=asfarray(b,float)    
    z_0=a[3]-a[2]   #overburden in [m]
    if z_0 < 4: #if the overburden is < 4, stop calculation (-> Tagbautunnel)
        continue
    #calculate normalvector for settlement
    ab = array([ abs(b[0]-a[0]),abs(b[1]-a[1]) ])   #vector between point i and i+1
    n1 = 1.0  
    n2 =( ab[0]*n1) / (ab[1])
    n_length = (n1**2 + n2**2)**0.5   #calculate length of normalvector
    n0 = array([n1,n2]) / n_length     #normalvector with length 1.0
    resolution = 2.0
    x_dir = arange(-40,40,resolution) #vector for calculating settlements 
    
    x = n0[0]*x_dir   #vector in GIS System
    
    xi = a[0]+n0[0]*x_dir   #x-coordinates of vector for GIS map
    yi = a[1]+n0[1]*x_dir   #y-coordinates of vector for GIS map
    
    x_0 = array([xi,yi])     #x- and y- coordinates of vector 
    
  
    '''Calculate Greenfield-Settlements with Gauss Kurve'''
    #Input Paraneter
    D_t=13.0    #Tunneldiameter in [m]                                                       #mv to input above
    K_g=0.5   #K_g [0.4-0.6] nach Fillibeck für nichtbindigen Boden, dicht gelagert
    K_t=0.7   #K_t [0.5-0.9] nach Fillibeck für bindigen Boden, dicht gelagert
    
    if a[4] < 0.0 :                                                                          #a[4]: rockcover
        i_0=K_g*z_0   #Wendepunktabstand für Tunnel im Lockergestein                         #z_0 : total cover
    else:                                                                                    #elif a[4] > 0.0 :
        i_0=K_t * a[4] + K_g*(z_0-a[4])   #Wendepunktabstand für Tunnel im Fels

    A_t = ((D_t**2.0) * pi) / 4.0
    V_l= 0.0064*(A_t/z_0)**(-0.75)  #nach Fillibeck,90%-Wahrscheinlichkeit, gilt für h/d>0.3

    S= -((pi/2)**(0.5)) * (V_l*(D_t**2))/(4*i_0) *exp(-(x_dir**2)/(2*i_0**2))   #calculate Settlement trough with gaussian curve
    beta=zeros((len(S)))    #creates vector for tangentenneigung
    beta[1:]=(diff(S)/resolution)  #tangentenneigung der setzungen

    
    '''classifydamage classes'''
    #classify beta into different damage classes
    dc_beta=chararray((len(S)),itemsize=3)
    dc_beta[abs(beta)<=(1./1000.)] = 'DC1'
    dc_beta[((1./1000.)<abs(beta)) & (abs(beta)<(1./500.))] = 'DC2'
    dc_beta[abs(beta)>=(1./500.)] = 'DC3'
    
    #classify settlements into different damage classes
    dc_s=chararray((len(S)),itemsize=3)
    dc_s[abs(S*1000.)<=5.] = 'DC1'
    dc_s[(5< abs(S*1000.)) & (abs(S*1000.)<25.)] = 'DC2'
    dc_s[abs(S*1000.)>=25.] = 'DC3'
    
    #classify settlements and beta into different damage classes
    dc=chararray((len(S)),itemsize=3)
    dc[ (abs(beta)<=1./500.) | (abs(S*1000)<=5.)] = 'DC1'
    dc[ ((1./500.<abs(beta)) & (abs(beta)<1./250.)) | ((5<abs(S*1000)) & (abs(S*1000)<25.))] = 'DC2'
    dc[ (abs(beta)>=1./250.) | (abs(S*1000)>=25.)] = 'DC3'
    
    '''data frame'''
    #create dataframe including the calculated settlements etc. 
    D= {'A. Point Coordinates x':a[0],
        'B. Point Coordinates y':a[1],
        'C. Normal Vector x': x_0[0],
        'D. Normal Vector y': x_0[1],
        'E. Settlement [mm]': S*1000,
        'F. Tangentenneigung': beta,
        'G. DC_Beta': dc_beta,
        'H. DC_Settlement': dc_s,
        'I. DamageClass' : dc,
        'J. z_0': z_0,}
    
    DF = pd.DataFrame(D)

    #append the calculated values to the dataframe
    DF_all=DF_all.append(DF)
    
#when loop is finished, safe dataframe to .csv
DF_all.to_csv('Settlement.csv')
print('run complete')

run complete


In [18]:
print(tv_df)

          Station      Easting     Northing  Elevation      DTM  RockSurface  \
0       204+874.1  2612071.548  1269294.122    270.962  253.524      236.000   
1         204+876  2612072.991  1269295.359    270.989  253.612      236.000   
2         204+879  2612075.303  1269297.270    271.033  253.160      236.000   
3         204+882  2612077.657  1269299.129    271.076  253.160      235.815   
4         204+885  2612080.052  1269300.936    271.120  252.934      235.815   
5         204+888  2612082.487  1269302.689    271.163  252.934      235.815   
6         204+891  2612084.959  1269304.388    271.207  252.934      235.815   
7         204+894  2612087.469  1269306.031    271.250  252.574      235.815   
8         204+897  2612090.014  1269307.619    271.294  252.013      235.815   
9         204+900  2612092.594  1269309.150    271.337  252.013      235.707   
10        204+903  2612095.208  1269310.623    271.381  251.805      235.707   
11        204+906  2612097.853  1269312.

In [19]:
# empirical calculation of settlement trough over tunnel (Gauss curve approach)
#                                                                                          WBS: 'PM'
#                                                                                          WBS: 'Feature'

# inputs                                                                                   #ToDo: mv to input above    
tunn_diam = 13.0  #tunnel diameter in m
tunn_area = 3.14159*(tunn_diam/2)**2
tunn_depth = 13.0  #distance between ground surface and tunnel centerline

# empirical estimates of calculation parameters
c_vol_loss = array([0.002, 0.004]) #empirical values for lost ground as fraction of excavation area
c_infl_len = array([0.8, 0.4]) #empirical coeff's to estimate distance centerline to inflection point of gauss curve

# statistical estimates of calculation parameters
# placeholder

# backcalculation estimates of calculation parameters
# placeholder

# calculate intermediate results
vol_lost = lambda tunn_area, c_vol_loss: (
    c_vol_loss*tunn_area)
infl_len = lambda tunn_depth, c_infl_len: (
    c_infl_len*tunn_depth) 
# calculte settlement at centerline
setl_max = lambda vol_lost, infl_len: (
    vol_lost / (sqrt(2*3.14259)*infl_len) )
# calculate settlment at distance x along gauss curve
setl = lambda x, setl_max, infl_len: (
    setl_max * exp(- x**2 / (2*infl_len**2))) 

# results
print('lost volume (m3): '+str(vol_lost(tunn_area, c_vol_loss)))
print('inflection length (m): '+str(infl_len(tunn_depth, c_infl_len)))
print('max settlement (m): '+str(setl_max(vol_lost(tunn_area, c_vol_loss), infl_len(tunn_depth, c_infl_len))))
x=10.4
print('settlement (m) at: '+str(x)+'m = '+str(
        setl(x, setl_max(vol_lost(tunn_area, c_vol_loss), infl_len(tunn_depth, c_infl_len)),
               infl_len(tunn_depth, c_infl_len))))

#    beta=zeros((len(S)))    #creates vector for tangentenneigung
#    beta[1:]=(diff(S)/resolution)  #tangentenneigung der setzungen

lost volume (m3): [ 0.26546436  0.53092871]
inflection length (m): [ 10.4   5.2]
max settlement (m): [ 0.01018155  0.04072621]
settlement (m) at: 10.4m = [ 0.00617542  0.00551169]


In [20]:
# empirical calculation of settlement trough over tunnel (Gauss curve approach)

# inputs                                                                                   #ToDo: mv to input above    
D = 13.0  #tunnel diameter in m
A = pi*(tunn_diam/2)**2
d = 13.0  #distance between ground surface and tunnel centerline

# empirical estimates of calculation parameters
c_v = array([0.002, 0.004]) #empirical values for lost ground as fraction of excavation area
c_i = array([0.8, 0.4]) #empirical coeff's to estimate distance centerline to inflection point of gauss curve

# statistical estimates of calculation parameters
# placeholder

# backcalculation estimates of calculation parameters
# placeholder

# calculate intermediate results
vol_lost = lambda A, c_v: (A*c_v)                               #ToDo:replace 3 intermediate results with variables
infl_len = lambda d, c_l: (d*c_i)
# calculte settlement at centerline
setl_max = lambda vol_lost, infl_len: (
    vol_lost / (sqrt(2*pi)*infl_len) )
# calculate settlment at distance x along gauss curve
setl = lambda x, setl_max, infl_len: (
    setl_max * exp(- x**2 / (2*infl_len**2))) 

# write results
print('lost volume (m3): '+str(vol_lost(A, c_v)))
print('inflection length (m): '+str(infl_len(d, c_i)))
print('max settlement (m): '+str(setl_max(vol_lost(A, c_v), infl_len(d, c_i))))
x = infl_len(d, c_i)[0]
print('settlement (m) at: '+str(x)+'m = '+str(
        setl(x, setl_max(vol_lost(A, c_v), infl_len(D, c_i)),
               infl_len(D, c_i))))

# plot results
x = arange(-40,42,2)
setl_rslt = []
y_data1 = []
y_data2 = []

for i in x:
    setl_rslt.append(setl(i, setl_max(vol_lost(A, c_v), infl_len(D, c_i)),
               infl_len(D, c_i)))
for i in setl_rslt:
    y_data1.append(-i[0])
    y_data2.append(-i[1])

for i in x:
    setl_rslt.append(setl(i, setl_max(vol_lost(A, c_v), infl_len(D, c_i)),
               infl_len(D, c_i)))
for i in setl_rslt:
    y_data1.append(-i[0])
    y_data2.append(-i[1])

trace0 = go.Scatter(
    x=x,
    y=y_data1
)
trace1 = go.Scatter(
    x=x,
    y=y_data2
)

data1 = Data([trace0,trace1])
plotly.offline.iplot(data1)

lost volume (m3): [ 0.26546458  0.53092916]
inflection length (m): [ 10.4   5.2]
max settlement (m): [ 0.01018318  0.04073271]
settlement (m) at: 10.4m = [ 0.00617641  0.00551257]


In [21]:
# empirical calculation of settlement trough over tunnel (Gauss curve approach)
#                                                                                          WBS: 'PM'
#                                                                                          WBS: 'Feature'

# inputs                                                                                   #ToDo: mv to input above    
D = 13.0  #tunnel diameter in m
A = pi*(tunn_diam/2)**2
d = 13.0  #distance between ground surface and tunnel centerline

# empirical estimates of calculation coefficients (they must have save number of items)
c_v = array([0.002, 0.004]) #empirical values for lost ground as fraction of excavation area
c_i = array([0.8, 0.4]) #empirical coeff's to estimate distance centerline to inflection point of gauss curve

# statistical estimates of calculation parameterspi
# placeholder

# backcalculation estimates of calculation parameters
# placeholder

# calculate intermediate results (d assumed constant along cross-section)
vol_loss = A * c_v
infl_len = d * c_i
# calculte settlement at centerline
setl_max = vol_lost / (sqrt(2*pi) * infl_len)
# calculate settlment at distance x along gauss curve                          #ToDo: lambda or function preferred
setl = lambda x, setl_max, infl_len: (
    setl_max * exp(- x**2 / (2*infl_len**2))) 

# write intermediate results
print('lost volume (m3): '+str(vol_loss))
print('inflection length (m): '+str(infl_len))
print('max settlement (m): '+str(setl_max))
#x=10.4  #for checking
#print('settlement (m) at: '+str(x)+'m = '+str(
#        setl(x, setl_max, infl_len)))

# calculate settlment at x along cross-section
x = arange(-40,41,1)
setl_rslt = []

for i in x:
    setl_rslt.append(setl(i, setl_max, infl_len))

# plot results
nr_rslts = len(setl_rslt[0])
y_data = [[] for n in range(nr_rslts)]
trace = [[] for n in range(nr_rslts)]
data = [[] for n in range(nr_rslts)]

for n in range(nr_rslts):
    for i in setl_rslt:
        y_data[n].append(-i[n])
    trace[n] = go.Scatter(
        x=x,
        y=y_data[n]
    )

data = Data(trace)
plotly.offline.iplot(data)

TypeError: unsupported operand type(s) for /: 'function' and 'float'

In [ ]:
# plotly examples

trace0 = Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = Data([trace0, trace1])

plotly.offline.iplot(data)

In [ ]:
trace1 = go.Scatter(x=[1,2,3], y=[4,5,6], marker={'color': 'red', 'symbol': 104, 'size': "10"}, 
                    mode="markers+lines",  text=["one","two","three"], name='1st Trace')
                                               
data=go.Data([trace1])
layout=go.Layout(title="First Plot", xaxis={'title':'x1'}, yaxis={'title':'x2'})
figure=go.Figure(data=data,layout=layout)
plotly.offline.iplot(figure, filename='pyguide_1')

In [ ]:
# Create random data with numpy
import numpy as np

N = 500
random_x = np.linspace(0, 1, N)
random_y = np.random.randn(N)

# Create a trace
trace = go.Scatter(
    x = random_x,
    y = random_y
)

data = [trace]

plotly.offline.iplot(data, filename='basic-line')

In [ ]:
# plot results - plan view
    #fig=plt.figure()
    #p1 = plt.plot(x_dir, S*1000) 
        #plt.xticks(y_pos+width, objects)
        #plt.ylim((0,110))
    #p2 = plt.plot(x_dir, a[2])
    #p2 = plt.plot(x_dir, a[3])
    #p3 = plt.plot(x_dir, a[4])
    #plt.ylabel('Settlement [mm]', **axis_font)
    #plt.xlabel('Distance from tunnel axis [m]')
    #plt.yticks( **axis_font)
    #plt.xticks( **axis_font)
    #titleA=('Greenfield-Settlements, Station ' +str(tv_df['Station'][i]))
    #plt.title(titleA, **title_font)
    #plt.legend()
    #plt.show()
    #fig.savefig('C:\Users\kzq653\Documents\geopython\settlementtrough_bilder_test_xxx.png', dpi=300)

In [ ]:
# plot results - cross-section
"""
    fig=plt.figure()
    p1 = plt.plot(x_dir, S*1000) 
        #plt.xticks(y_pos+width, objects)
        #plt.ylim((0,110))
    p2 = plt.plot(x_dir, a[2])
    p2 = plt.plot(x_dir, a[3])
    p3 = plt.plot(x_dir, a[4])
    plt.ylabel('Settlement [mm]', **axis_font)
    plt.xlabel('Distance from tunnel axis [m]', **axis_font)
    plt.yticks( **axis_font)
    plt.xticks( **axis_font)
    titleA=('Greenfield-Settlements, Station ' +str(a[5]))
    plt.title(titleA, **title_font)
    plt.legend()
    plt.show()
    fig.savefig('C:\Users\kzq653\Documents\geopython\settlementtrough_bilder_test' + str(titleA) +'.png', dpi=300)
"""

In [ ]:
help(ff.create_gantt)

In [22]:
    import plotly.plotly as py
    from plotly.figure_factory import create_gantt
    
    # Make data for chart
    df = [dict(Task="Job A", Start='2009-01-01', Finish='2009-02-30'),
          dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15'),
          dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30')]
    
    # Create a figure
    fig = create_gantt(df)
    
    # Plot the data
    plotly.offline.iplot(fig)

In [23]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    y=['giraffes', 'orangutans', 'monkeys'],
    x=[20, 14, 23],
    name='SF Zoo',
    orientation = 'h',
    marker = dict(
        color = 'rgba(246, 78, 139, 0.6)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',
            width = 3)
    )
)
trace2 = go.Bar(
    y=['giraffes', 'orangutans', 'monkeys'],
    x=[12, 18, 29],
    name='LA Zoo',
    orientation = 'h',
    marker = dict(
        color = 'rgba(58, 71, 80, 0.6)',
        line = dict(
            color = 'rgba(58, 71, 80, 1.0)',
            width = 3)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='marker-h-bar')

In [24]:
# Plotyl Example: Color Palette for Bar Chart
#   https://plot.ly/python/horizontal-bar-charts/

import plotly.plotly as py
import plotly.graph_objs as go

# initialize variables 
x_data = []
y_data = []
traces = []
annotations = []

top_labels = ['Strongly<br>agree', 'Agree', 'Neutral', 'Disagree',
              'Strongly<br>disagree']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)']

x_data = [[21, 30, 21, 16, 12],
          [24, 31, 19, 15, 11],
          [27, 26, 23, 11, 13],
          [29, 24, 15, 18, 14]]

y_data = ['The course was effectively<br>organized',
          'The course developed my<br>abilities and skills ' +
          'for<br>the subject', 'The course developed ' +
          'my<br>ability to think critically about<br>the subject',
          'I would recommend this<br>course to a friend']

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
#            x=xd[i],
#            y=yd,
            x=[xd[i]],                                                    #x,y must be array (also if only 1 item)
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))
        
layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
plotly.offline.iplot(fig, filename='bar-colorscale')